In [7]:
import json
import re
from copy import deepcopy
from collections import OrderedDict
from nltk.tokenize import wordpunct_tokenize as tagger

# Frame 

In [2]:
data = json.load(open('data/frames.json','r',encoding='utf-8'))

In [3]:
data[0]['turns'][2]

{'author': 'user',
 'labels': {'active_frame': 2,
  'acts': [{'args': [{'key': 'dst_city', 'val': 'Neverland'},
     {'key': 'n_adults', 'val': '5'},
     {'key': 'budget', 'val': '1900'},
     {'key': 'ref',
      'val': [{'annotations': [{'key': 'or_city', 'val': 'Caprica'},
         {'key': 'str_date', 'val': 'August 13, 2016'}],
        'frame': 1,
        'fromrange': False}]}],
    'name': 'inform'}],
  'acts_without_refs': [{'args': [{'key': 'dst_city', 'val': 'Neverland'},
     {'key': 'n_adults', 'val': '5'},
     {'key': 'budget', 'val': '1900'},
     {'key': 'or_city', 'val': 'Caprica'},
     {'key': 'str_date', 'val': 'August 13, 2016'}],
    'name': 'inform'}],
  'frames': [{'binary_questions': [],
    'compare_requests': [],
    'frame_id': 1,
    'frame_parent_id': None,
    'info': {'NO_RESULT': [{'negated': False, 'val': True}],
     'budget': [{'negated': False, 'val': '1700.0'}],
     'dst_city': [{'negated': False, 'val': 'Atlantis'}],
     'intent': [{'negated': Fa

In [4]:
dialogs=[]

In [5]:
for d in data:
    dialog=[]
    for turn in d['turns']:
        if turn['author']=='wizard': continue
        temp=[]
        temp.append(turn['text'])
        temp.append(turn['labels']['acts'])
        dialog.append(temp)
    dialogs.append(dialog)

In [27]:
dialogs_p=[]

In [28]:
for dialog in dialogs:
    temp=[]
    for pair in dialog:
        try:
            raw = deepcopy(pair[0])
            bio = ['O']*len(tagger(raw))

            for slot in pair[1]:
                if slot['name'] in ['inform','switch_frame']:
                    for ss in slot['args']:
                            if ss['key']!='intent':
                                value = ss['val']
                                tag = ss['key']
                                if type(value)==list and tag=='ref':
                                    for sss in ss['val'][0]['annotations']:
                                        if 'val' not in sss.keys() or type(sss['val'])!=str:
                                            continue

                                        value = sss['val']
                                        tag = sss['key']
                                        raw = deepcopy(pair[0])
                                        raw = raw.replace(value,' '.join(['19191919']*len(tagger(value))))
                                        seq = tagger(raw)
                                        start=False
                                        for i,s in enumerate(seq):
                                            if s=='19191919':
                                                if start==False:
                                                    bio[i]='B-'+tag
                                                    start=True
                                                else:
                                                    bio[i]='I-'+tag
                                                seq[i]='@'

                                elif type(value)==str:        
                                    raw = deepcopy(pair[0])
                                    raw = raw.replace(value,' '.join(['19191919']*len(tagger(value))))
                                    seq = tagger(raw)
                                    start=False
                                    for i,s in enumerate(seq):
                                        if s=='19191919':
                                            if start==False:
                                                bio[i]='B-'+tag
                                                start=True
                                            else:
                                                bio[i]='I-'+tag
                                            seq[i]='@'

            temp.append([tagger(pair[0]),bio])
        except:
            continue
    dialogs_p.append(temp)

In [13]:
len(dialogs_p)

1369

In [33]:
with open('frame.iob','a',encoding='utf-8') as f:
    for dialog in dialogs_p:
        for pair in dialog:
            f.write(' '.join(pair[0])+'\t'+' '.join(pair[1])+'\n')
        f.write('\n')

In [ ]:
def make_iob(phrase,goal_tracker):
        phrase = deepcopy(phrase)
        goal_tracker = deepcopy(goal_tracker)
        if len(self.global_user_goal)>0: goal_tracker.update(self.global_user_goal)
        order = [[(k,v),phrase.index(v)] for k,v in goal_tracker.items() if v !=None and v in phrase]
        order = sorted(order, key=lambda x: x[1])
        goal_tracker = OrderedDict([o[0] for o in order])
        pos = tagger(phrase)
        tag=["O"]*len(pos)
        for k,v in goal_tracker.items():
            B=True
            if v in phrase:
                for i,p in enumerate(pos):
                    if p!="<MASK>" and p in v:
                        if B:
                            tag[i] = 'B-'+k
                            B=False
                            pos[i]="<MASK>"
                        else:
                            if i!=0 and tag[i-1][0] in ['B','I']: 
                                tag[i] = 'I-'+k
                                pos[i]="<MASK>"
        return tag

# FRAME wizard

In [1]:
import json

In [2]:
data = json.load(open('./Conv_data/Frames-dataset/frames.json','r',encoding='utf-8'))

In [3]:
dialogs=[]

In [19]:
data[0]['turns'][3]['labels']

{'active_frame': 2,
 'acts': [{'args': [{'key': 'ref',
     'val': [{'annotations': [{'key': 'str_date', 'val': None}],
       'frame': 2,
       'fromrange': False}]}],
   'name': 'no_result'},
  {'args': [{'key': 'str_date'}, {'key': 'end_date'}], 'name': 'suggest'}],
 'frames': [{'binary_questions': [],
   'compare_requests': [],
   'frame_id': 1,
   'frame_parent_id': None,
   'info': {'NO_RESULT': [{'negated': False, 'val': True}],
    'budget': [{'negated': False, 'val': '1700.0'}],
    'dst_city': [{'negated': False, 'val': 'Atlantis'}],
    'intent': [{'negated': False, 'val': 'book'}],
    'n_adults': [{'negated': False, 'val': '8'}],
    'or_city': [{'negated': False, 'val': 'Caprica'}],
    'str_date': [{'negated': False, 'val': 'august 13'}]},
   'requests': []},
  {'binary_questions': [],
   'compare_requests': [],
   'frame_id': 2,
   'frame_parent_id': 1,
   'info': {'NO_RESULT': [{'negated': False, 'val': True}],
    'budget': [{'negated': False, 'val': '1900.0'}],
    

In [36]:
data[1]['turns'][17]['text']

'There are no activities listed for this package. You will be staying at the Ivory Legacy Hotel for 7 days. This hotel has a 4.5 star rating and free wifi. Will I go ahead and book this package?'

In [37]:
data[1]['turns'][18]['text']

'Yes, I would like to book this package.'

In [ ]:
for d in data:
    dialog=[]
    for turn in d['turns']:
        if turn['author']=='wizard': continue
        temp=[]
        temp.append(turn['text'])
        temp.append(turn['labels']['acts'])
        dialog.append(temp)
    dialogs.append(dialog)